This building model is a simplified faster version of the full two-zone building model. It assumes the building is always at thermal equilibrium and computes the duty cycle required to maintain the setpoint. 

# Building dynamics
Buildings can be simulated using a reduced-order model using a discrete transfer function based on the thermal properties of the building.  The transfer function can be derived for a two-variable model using the indoor air temperature $T_A$ and building mass temperature $T_M$ as they respond to changes in outdoor air temperature $T_O$ and changes in heat gains to the air $Q_A$ and mass $Q_M$, i.e.,
$$
    C_A \dot T_A = U_A ( T_O - T_A ) + U_I ( T_M - T_A ) + Q_A \qquad (1)
$$
$$
    C_M \dot T_M = U_I (T_A - T_M ) + U_M ( T_O - T_M ) + Q_M \qquad (2)
$$
where
* $C_A$ is the heat capacity of the air volume;
* $C_M$ is the heat capacity of the building mass;
* $U_A$ is the conductance between the indoor to outdoor air;
* $U_I$ is the conductance between the indoor air and the building mass,; and
* $U_M$ is the conductance between the building mass and the outdoor air.

The heat gains to the air come from the following sources:
* $S_F E$, the internal enduse electric loads, where $S_F$ is the floor area and $E$ is the  electric end-use load density;
* $S_F G$, the internal enduse gas loads, $G$ is the gas end-uses load density;
* $Q_O H$, the occupancy heat gains where $Q_O$ is the heat gain for a single person, and $H$ is the number of people present;
* $Q_V H$, the ventilation heat gains, where $Q_V$ is the heat gain for fresh air per person; and
* $M Q_H$, the HVAC heat gains, where $M$ is the system mode, and $Q_H$ is the nameplate system capacity.
Thus
$$
    Q_A = E Q_E + G Q_G + H (Q_O+Q_V) + M Q_H
$$

The heat gains to the mass from the solar gains to building envelope and mass through windows, such that $Q_M = S_A Q_S$, where $S_A$ is the solar exposure area of the building mass, and $Q_S$ is the total insolation.

The HVAC system mode is included as a state variable such that the mode $M$ represents the state of the HVAC system given the indoor air temperature setpoint $T_S$ deviation from $T_A$ with a proportional gain $K$, where $M>0$ for heating, and $M<0$ for cooling. 

# System Sizing

If the system size is not given, then $Q_H$ must be calculated for the design conditions.  The heating system capacity is found by solving Eq. (3) at steady-state for $Q_H$ given the $T_O=T_H$, $T_A=T_S$, $E=G=H=Q_S=0$, and $M=1/D_F$, the system over-design factor. This gives us the equation
$$
    \left[ \begin{matrix} 0 \\ 0 \end{matrix} \right]
    =
    \left[ \begin{matrix} 
        {U_I \over C_A} & {1 \over D_F C_A} \\
        -{U_M+U_I \over C_M} & 0
    \end{matrix} \right]
    \left[ \begin{matrix} T_M \\ Q_H \end{matrix} \right]
    +
    \left[ \begin{matrix} 
        {U_A \over C_A} & -{U_A+U_I \over C_A} \\
        {U_M \over C_M} & {U_I \over C_M}
    \end{matrix} \right]
    \left[ \begin{matrix} T_H \\ T_S \end{matrix} \right],
$$
which we solve for $Q_H$.
The same method is used for cooling but with all internal gains on and maximum solar gain.
$$
    \left[ \begin{matrix} 0 \\ 0 \end{matrix} \right]
    =
    \left[ \begin{matrix} 
        {U_I \over C_A} & {1 \over D_F C_A} \\
        -{U_M+U_I \over C_M} & 0
    \end{matrix} \right]
    \left[ \begin{matrix} T_M \\ Q_C \end{matrix} \right]
    +
    \left[ \begin{matrix} 
        {U_A \over C_A} & -{U_A+U_I \over C_A} & 0 \\
        {U_M \over C_M} & {U_I \over C_M} & { Q_E+Q_G+Q_O+Q_V+1300S_A \over C_M }
    \end{matrix} \right]
    \left[ \begin{matrix} T_H \\ T_S \\ 1 \end{matrix} \right].
$$
The required system capacity is the greater of $Q_H$ and $-Q_C$.

# Thermal model
The state space model is thus
$$
    \left[ \begin{matrix} \dot T_A \\ \dot T_M \\ \dot M \end{matrix} \right]
    =
    \left[ \begin{matrix} 
        -{U_A+U_I \over C_A} & {U_I \over C_A} & {Q_H \over C_A}
    \\
        {U_I \over C_M} & -{U_M+U_I \over C_M} & 0
    \\
        -K & 0 & 0
    \end{matrix} \right]
    \left[ \begin{matrix} T_A \\ T_M \\ M \end{matrix} \right]
    +
    \left[ \begin{matrix} 
        { U_A \over C_A } & {S_F \over C_A} & {S_F \over C_A} & {Q_O+Q_V \over C_A} & 0 & 0
    \\
        { U_M \over C_M } & 0 & 0 & 0 & {S_A \over C_M} & 0
    \\
        0 & 0 & 0 & 0 & 0 & K
    \end{matrix} \right]
    \left[ \begin{matrix} T_O \\ E \\ G \\ H \\ Q_S \\ T_S \end{matrix} \right].
    \qquad(3)
$$


# Equilibrium Conditions
The equilibrium condition $\bar x$ is computed by solving for $\dot x = 0$, i.e.,
$$
    \bar x = -A^{-1} B u,
$$
the result of include the duty cycle $\bar M$ required to maintain the indoor air temperature setpoint.

# Inputs
The inputs are obtain from climate ($T_O$ and $Q_S$), electric and gas loadshapes ($E$ and $G$), occupancy ($H$), and the thermostat setpoint ($T_S$).

## Electrification 
The electric and gas loadshapes are computed from the enduses as follows:
$$
    E = \left( E_U + \frac{E_F N_G}{E_E} \right) E_G
$$
and
$$
    G = (1-E_F) N_G G_G,
$$
where
* $E_U$ is the electric end-use load density `W/sf`,
* $E_F$ is the electrification fraction `pu`,
* $N_G$ is the natural gas load density `W/sf`,
* $E_E$ is the efficiency of electric enduses relative to gas enduses `pu`,
* $E_G$ is the electric enduse heat gain fraction `pu`, and
* $G_G$ is the gas enduse heat gain fraction `pu`.

## Occupancy
The occupancy is computed as $H = N_H H_O$, where
* $N_H$ is the nominal building occupancy, and
* $H_O$ is the fractional building occupancy occupancy `pu`.

Occupancies schedules for building types and day types are 

| building_type | daytype | start | stop | occupied | unoccupied |
| ------------- | ------- | ----- | ---- | -------- | ---------- |
| APARTMENT | WEEKDAY | 8 | 18 | 0.2 | 1.0 |
| APARTMENT | WEEKEND | 0 | 24 | 1.0 | 0.0 |
| CONDO | WEEKDAY | 8 | 18 | 0.2 | 1.0 |
| CONDO | WEEKEND | 0 | 24 | 1.0 | 0.0 |
| EDUCATION | WEEKDAY | 7 | 16 | 1.0 | 0.0 |
| EDUCATION | WEEKEND | 0 | 24 | 0.0 | 0.0 |
| GROCERY | WEEKDAY | 8 | 20 | 1.0 | 0.0 |
| GROCERY | WEEKEND | 8 | 20 | 1.0 | 0.0 |
| HOUSE | WEEKDAY | 8 | 18 | 0.2 | 1.0 |
| HOUSE | WEEKEND | 0 | 24 | 1.0 | 0.0 |
| HEALTHCARE | WEEKDAY | 6 | 22 | 1.0 | 0.5 |
| HEALTHCARE | WEEKEND | 6 | 22 | 1.0 | 0.5 |
| LARGEOFFICE | WEEKDAY | 9 | 17 | 1.0 | 0.0 |
| LARGEOFFICE | WEEKEND | 0 | 24 | 0.1 | 0.0 |
| LODGING | WEEKDAY | 6 | 8 | 1.0 | 0.5 |
| LODGING | WEEKEND | 5 | 9 | 1.0 | 0.5 |
| RESTAURANT | WEEKDAY | 10 | 20 | 1.0 | 0.0 |
| RESTAURANT | WEEKEND | 10 | 20 | 1.0 | 0.0 |
| RETAIL | WEEKDAY | 10 | 18 | 1.0 | 0.0 |
| RETAIL | WEEKEND | 10 | 18 | 1.0 | 0.0 |
| SMALLOFFICE | WEEKDAY | 9 | 17 | 1.0 | 0.0 |
| SMALLOFFICE | WEEKEND | 0 | 24 | 0.1 | 0.0 |
| TOWNHOUSE | WEEKDAY | 8 | 18 | 0.2 | 1.0 |
| TOWNHOUSE | WEEKEND | 0 | 24 | 1.0 | 0.0 |

where $start$ and $stop$ are the occupied period start and stop hours, respectively.  The $occupied$ and $unoccupied$ values are given per-unit of nominal building capacity.

# Output
The real and reactive ZIP components, $\mathbf P$ and $\mathbf Q$, respectively, are obtained from the enduse loads $Q_E$ and the state of the HVAC system $M$, including the system efficiency $\eta$, and the real and reactive power load compositions:
$$
    \left[ \begin{matrix} P_Z \\ P_I \\ P_P \\ Q_Z \\ Q_I \\ Q_P \end{matrix} \right]
    =
    \left[ \begin{matrix}
        0 & 0 & P_{Z_M} Q_C
    \\
        0 & 0 & 0
    \\
        0 & 0 & P_{P_M} Q_C
    \\
        0 & 0 & 0
    \\
        0 & 0 & 0 
    \\
        0 & 0 & Q_{P_M} Q_C
    \end{matrix} \right] 
    \left[ \begin{matrix} T_A \\ T_M \\ M \end{matrix} \right]
    +
    \left[ \begin{matrix}
        0 & S_F P_{Z_E} & 0 & 0 & 0 & 0 
    \\
        0 & S_F P_{I_E} & 0 & 0 & 0 & 0 
    \\
        0 & S_F P_{P_E} & 0 & 0.1 P_{P_H} Q_H & -P_V & 0 
    \\
        0 & S_F Q_{Z_E} & 0 & 0 & 0 & 0 
    \\
        0 & S_F Q_{I_E} & 0 & 0 & 0 & 0 
    \\
        0 & S_F Q_{P_E} & 0 & 0.01 Q_{P_H} Q_H & 0 & 0 
    \end{matrix} \right] 
    \left[ \begin{matrix} T_O \\ E \\ G \\ H \\ Q_S \\ T_S \end{matrix} \right]
$$
where
$$
    Q_C = \left\{ \begin{matrix}
        Q_H &:& M \lt 0
    \\
        -Q_H &:& M \ge 0
    \end{matrix} \right.
$$
The load $y$ is divided equally across the phases to which the building is connected such that the base power $P_0 = P / N_{phases}$ where $P$ is the total real power. Note that the magnitude of the base power export may be not exceed the export limit $P_X$. 

## Distributed Generation
Solar generation contribution to distributed generation is
$$
    D_G = P_V Q_S
$$

## Energy Storage
Energy storage is modeled only when $B_S>0$.  The power available to store or discharge is 
$$
    R = \left\{ \begin{array}{lll}
        P_G > 0 \quad\mathrm{and}\quad P_P < 0 &:& -\min(-P_P,P_G)
    \\
        P_G > 0 \quad\mathrm{and}\quad P_P \ge 0 &:& \min(P_P,P_G)
    \\
        P_G \le 0 &:& P_P
    \end{array} \right.
$$
where $P_G$ is the inverter power capacity.

When exporting power and able to store power, i.e., $P_P < 0$ and $E_S < B_S$, where $B_S$ is the maximum storage possible, and $E_S$ is the current storage, excess power is be stored subject to the constraint $R$ and remaining capacity $B_S-E_S$. Thus
$$
    E_S = \left\{ \begin{array}{lll}
        B_S &:& -P_P~dt > \min(B_S-E_S,-R~dt)
    \\
        E_S - P_P ~ dt &:& otherwise
    \end{array} \right.
$$
and
$$
    D_S = \min\left(\frac{B_S-E_S}{dt},-R\right)
$$

When no power can be exported and stored energy is available, i.e., $P_P > 0$ and $E_S > 0$, the load is served subject to the available power and energy capacity. Thus
$$
    E_S = E_S - \min\left(\frac{E_S}{dt},r\right)
$$
and
$$
    D_S = \left\{ \begin{array}{lll}
        - \min\left(\frac{E_S}{dt},r\right) :&: P_P > - \min\left(\frac{E_S}{dt},r\right)
    \\
        -P_P :&: otherwise
    \end{array} \right.
$$

The total storage $D_S$ is added to the real power power $P_P$ as well as the distributed generation $D_G$.

## Inverter Control
The inverter implements a parametric volt-var control strategy to support system voltage. The control parameters and strategies are as follows:

| Strategy | $V_{min}$ | $V_{max}$ | $Q_{min}$ | $Q_{max}$ | $S_{low}$ | $S_{high}$ |
| -------- | --------- | --------- | --------- | --------- | --------- | ---------- |
| UNITY    |      0.00 |      0.00 |      0.00 |      0.00 |      0.00 |       0.00 |
| IEEE1547 |      0.95 |      1.05 |     -0.10 |      0.10 |     -0.05 |      -0.05 |
| CARULE21 |      0.95 |      1.05 |     -0.10 |      0.10 |     -0.05 |      -0.05 |
| UL1741   |      0.95 |      1.05 |     -0.10 |      0.10 |     -0.05 |      -0.05 |

where $V_{min}$ and $V_{max}$ are the voltage deadband limits within which no VAR control is implemented, $S_{low}$, and $S_{high}$ are the slopes of the VAR control response, and $Q_{min}$ and $Q_{max}$ are the maximum VAR outputs.

The maximum control possible is $M=\max(|D_G|,P_G)$.

When the line voltage is below $V_{min}$, then
$$
    I_C = M \angle \sin^{-1} \left( \frac{R}{M} \right)
$$
where $R = \min((V-V_{min}) S_{low},Q_{max}) ~ m$.

When the line voltage is above $V_{max}$, then
$$
    I_C = M \angle \sin^{-1} \left( \frac{R}{M} \right)
$$
where $R = \max((V_{max}-V) S_{high},Q_{min}) ~ P_{max}$.
 

## Load Composition
The load composition is
$$
    \begin{array}{l}
        P_z = P_Z / P 
    \\ 
        P_i = P_I / P 
    \\ 
        P_p = P_P / P
    \\
        F_z = sign(Q_Z) P_Z / |P+jQ|
    \\
        F_i = sign(Q_I) P_I / |P+jQ|
    \\
        F_p = sign(Q_P) P_P / |P+jQ|
    \end{array}
$$
where
* $Q$ is the total reactive power,
* $P_z$, $P_i$, and $P_p$ are the fractional constant impedance, current, and power contributions to the load, and
* $F_z$, $F_i$, and $F_p$ are the power factors for the constant impedance, current, and power components, respectively.